## Dependencies for OpenAI 🦜️Langchain 🏓Pinecone

In [66]:
import os
import time
import dotenv
import openai
import getpass
import textwrap
import tempfile
import tiktoken

from openai import OpenAI
from docx import Document
from dotenv import load_dotenv
from tqdm.autonotebook import tqdm
from langchain_openai import ChatOpenAI
from typing import List, Tuple, Optional
from langchain.prompts import PromptTemplate
from pinecone import Pinecone, ServerlessSpec
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain.chains.mapreduce import MapReduceChain
from langchain import OpenAI, PromptTemplate, LLMChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import DirectoryLoader

#os.environ['USER_AGENT'] = 'myagent'

## Keys

In [67]:

load_dotenv()
# Fetching the API keys from environment variables
openai_api_key = os.getenv('OPENAI_API_KEY')
pinecone_api_key = os.getenv('PINECONE_API_KEY')


# Setting up the environmental variables
if openai_api_key:
    os.environ['OPENAI_API_KEY'] = openai_api_key
if pinecone_api_key:
    os.environ['PINECONE_API_KEY'] = pinecone_api_key  

#Verifications that the keys are loaded
# print(f"OpenAI API Key: {os.environ.get('OPENAI_API_KEY')}")
# print(f"Pinecone API Key: {os.environ.get('PINECONE_API_KEY')}")



## Vector Embeddings 

In [68]:
os.environ["HUGGINGFACEHUB_API_TOKEN"]="HUGGINGFACEHUB_API_TOKEN"
embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002",  #response time is 9s  #infloat/e5-base-V2 has 3.53sec response time.
)
embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x12dd51d30>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x12ddde5d0>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

## Pinecone Set Up

In [69]:
pc = Pinecone(api_key=pinecone_api_key)

index_name = "noveltyai"  

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)
#pc.list_indexes()

## Files Loader

In [86]:
directory = '/Users/user/Downloads/summarizer/directory'
def load_documents(directory):
    loader = DirectoryLoader(directory)
    documents = loader.load()
    return documents

documents = load_documents(directory)

## Text splitter (Chunks)

In [88]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)
text = text_splitter.split_documents(documents)

## Pinecone Initializations

In [89]:
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
vectorstore = PineconeVectorStore.from_documents(text, embeddings, index_name=index_name)

## OpenAI Initializations

In [95]:
# Initialize the ChatOpenAI module and load the summarize chain
llm = ChatOpenAI(temperature=0, openai_api_key=openai_api_key)
#chain = load_summarize_chain(llm, chain_type="stuff")
chain = load_summarize_chain(llm, chain_type="map_reduce")

## Similarity search

In [98]:
# Perform a similarity search on the vectorstore and summarize the relevant sections
search = vectorstore.similarity_search(" ")
summary = chain.run(input_documents=search, question="Write a concise summary within 250 words.", summarize_recursively = True)

/Users/user/.pyenv/versions/3.12.3/lib/python3.12/site-packages/langchain_openai/chat_models/base.py:357: UserWarning: Unexpected type for token usage: <class 'NoneType'>
  warnings.warn(f"Unexpected type for token usage: {type(new_usage)}")


## Summary

In [99]:
print(summary)

The system allows for the uploading, downloading, renaming, and viewing of files. Client ID cards can be viewed in the ID tab if enabled. Clients are initially marked as "Submitted" and will only be charged once approved. Dependents can be added to primary clients. Client details can be viewed by clicking on the + sign, showing membership, billing, and cases information. Groups or companies can be set up in the Group tab. Client Ellipsis functions allow for editing profile information, viewing membership status, updating billing details, and uploading files.


## Recursive Summary

In [116]:
summary = chain.run(input_documents=search, detail=1, question="Summarize the key details of the Admin and Client portal of Vitafy Healthcare system in a paragraph.", summarize_recursively = True)
print(summary)

/Users/user/.pyenv/versions/3.12.3/lib/python3.12/site-packages/langchain_openai/chat_models/base.py:357: UserWarning: Unexpected type for token usage: <class 'NoneType'>
  warnings.warn(f"Unexpected type for token usage: {type(new_usage)}")


The system allows for the uploading, downloading, renaming, and viewing of files. Client ID cards can be viewed in the ID tab if enabled. Clients are initially marked as "Submitted" and will only be charged once approved. Dependents can be added to primary clients, who are individuals connected through family ties or association. Client details, memberships, billing information, and recent cases can be viewed in the system. Groups or companies can be set up by filling out an online form with primary and dependent client types. Client Ellipsis functions allow for editing profile information, viewing membership status, updating billing details, and uploading files for specific clients.


## Tracking token & cost usage 


In [109]:
from langchain_community.callbacks import get_openai_callback

with get_openai_callback() as cb:
    summary = chain.run(input_documents=search, detail=1, question="Summarize the key details of the Admin and Client portal of Vitafy Healthcare system in a paragraph.", summarize_recursively = True)
    print(summary)  
print(cb)    

/Users/user/.pyenv/versions/3.12.3/lib/python3.12/site-packages/langchain_openai/chat_models/base.py:357: UserWarning: Unexpected type for token usage: <class 'NoneType'>
  warnings.warn(f"Unexpected type for token usage: {type(new_usage)}")


Clients can upload, download, rename, and view files. Client ID cards can be viewed in the ID tab if enabled. Clients are initially marked as "Submitted" and will only be charged once approved. Dependents can be added to primary clients. Client details can be viewed by clicking on the + sign. The billing section displays recent transactions. The Cases section shows recent cases associated with the client. Instructions are provided on setting up a group or company in the system. Client Ellipsis functions allow for editing profile information and managing membership status and billing details.
Tokens Used: 1007
	Prompt Tokens: 685
	Completion Tokens: 322
Successful Requests: 5
Total Cost (USD): $0.0008255000000000001


## For Very Longer Documents

In [114]:
# Function to read PDF, DOCX, and TXT files from a specified directory
def load_documents_from_directory(directory: str) -> str:
    all_text = ""
    
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        
        if filename.endswith(".txt"):
            # Read .txt files
            with open(file_path, "r", encoding="utf-8") as file:
                all_text += file.read() + "\n\n"
        
        elif filename.endswith(".pdf"):
            # Read .pdf files using PyPDFLoader
            loader = PyPDFLoader(file_path)
            pdf_pages = loader.load_and_split()
            pdf_text = "\n\n".join(page.page_content for page in pdf_pages)
            all_text += pdf_text + "\n\n"
        
        elif filename.endswith(".docx"):
            # Read .docx files using python-docx
            doc = Document(file_path)
            docx_text = "\n\n".join([para.text for para in doc.paragraphs if para.text.strip()])
            all_text += docx_text + "\n\n"
    
    return all_text

directory_path = "/Users/user/Downloads/summarizer/directory"  
text = load_documents_from_directory(directory_path)


In [115]:
# load encoding and check the length of dataset
encoding = tiktoken.encoding_for_model('gpt-4-turbo')
len(encoding.encode(text))

2379

## We'll define a simple utility to wrap calls to the OpenAI API

In [105]:
def get_chat_completion(messages, model='gpt-4-turbo'):
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message.content


## Next, we'll define some utilities to chunk a large document into smaller pieces

In [76]:
def tokenize(text: str) -> List[str]:
    encoding = tiktoken.encoding_for_model('gpt-4-turbo')
    return encoding.encode(text)


# This function chunks a text into smaller pieces based on a maximum token count and a delimiter.
def chunk_on_delimiter(input_string: str,
                        max_tokens: int, delimiter: str) -> List[str]:
    chunks = input_string.split(delimiter)
    combined_chunks, _, dropped_chunk_count = combine_chunks_with_no_minimum(
        chunks, max_tokens, chunk_delimiter=delimiter, add_ellipsis_for_overflow=True
    )
    if dropped_chunk_count > 0:
        print(f"warning: {dropped_chunk_count} chunks were dropped due to overflow")
    combined_chunks = [f"{chunk}{delimiter}" for chunk in combined_chunks]
    return combined_chunks


# This function combines text chunks into larger blocks without exceeding a specified token count. It returns the combined text blocks, their original indices, and the count of chunks dropped due to overflow.
def combine_chunks_with_no_minimum(
        chunks: List[str],
        max_tokens: int,
        chunk_delimiter="\n\n",
        header: Optional[str] = None,
        add_ellipsis_for_overflow=False,
) -> Tuple[List[str], List[int]]:
    dropped_chunk_count = 0
    output = []  # list to hold the final combined chunks
    output_indices = []  # list to hold the indices of the final combined chunks
    candidate = (
        [] if header is None else [header]
    )  # list to hold the current combined chunk candidate
    candidate_indices = []
    for chunk_i, chunk in enumerate(chunks):
        chunk_with_header = [chunk] if header is None else [header, chunk]
        if len(tokenize(chunk_delimiter.join(chunk_with_header))) > max_tokens:
            print(f"warning: chunk overflow")
            if (
                    add_ellipsis_for_overflow
                    and len(tokenize(chunk_delimiter.join(candidate + ["..."]))) <= max_tokens
            ):
                candidate.append("...")
                dropped_chunk_count += 1
            continue  # this case would break downstream assumptions
        # estimate token count with the current chunk added
        extended_candidate_token_count = len(tokenize(chunk_delimiter.join(candidate + [chunk])))
        # If the token count exceeds max_tokens, add the current candidate to output and start a new candidate
        if extended_candidate_token_count > max_tokens:
            output.append(chunk_delimiter.join(candidate))
            output_indices.append(candidate_indices)
            candidate = chunk_with_header  # re-initialize candidate
            candidate_indices = [chunk_i]
        # otherwise keep extending the candidate
        else:
            candidate.append(chunk)
            candidate_indices.append(chunk_i)
    # add the remaining candidate to output if it's not empty
    if (header is not None and len(candidate) > 1) or (header is None and len(candidate) > 0):
        output.append(chunk_delimiter.join(candidate))
        output_indices.append(candidate_indices)
    return output, output_indices, dropped_chunk_count

## Now, we can define a utility to summarize text with a controllable level of detail 

The function first determines the number of chunks by interpolating between a minimum and a maximum chunk count based on a controllable detail parameter. 
It then splits the text into chunks and summarizes each chunk.


In [79]:
def summarize(text: str,
                detail: float = 0,
                model: str = 'gpt-4-turbo',
                additional_instructions: Optional[str] = None,
                minimum_chunk_size: Optional[int] = 500,
                chunk_delimiter: str = ".",
                summarize_recursively=False,
                verbose=False):
    """
    Summarizes a given text by splitting it into chunks, each of which is summarized individually. 
    The level of detail in the summary can be adjusted, and the process can optionally be made recursive.

    Parameters:
    - text (str): The text to be summarized.
    - detail (float, optional): A value between 0 and 1 indicating the desired level of detail in the summary.
    0 leads to a higher level summary, and 1 results in a more detailed summary. Defaults to 0.
    - model (str, optional): The model to use for generating summaries. Defaults to 'gpt-3.5-turbo'.
    - additional_instructions (Optional[str], optional): Additional instructions to provide to the model for customizing summaries.
    - minimum_chunk_size (Optional[int], optional): The minimum size for text chunks. Defaults to 500.
    - chunk_delimiter (str, optional): The delimiter used to split the text into chunks. Defaults to ".".
    - summarize_recursively (bool, optional): If True, summaries are generated recursively, using previous summaries for context.
    - verbose (bool, optional): If True, prints detailed information about the chunking process.

    Returns:
    - str: The final compiled summary of the text.

    The function first determines the number of chunks by interpolating between a minimum and a maximum chunk count based on the `detail` parameter. 
    It then splits the text into chunks and summarizes each chunk. If `summarize_recursively` is True, each summary is based on the previous summaries, 
    adding more context to the summarization process. The function returns a compiled summary of all chunks.
    """

    # detail is set 
    assert 0 <= detail <= 1

    # interpolating the number of chunks based to get specified level of detail
    max_chunks = len(chunk_on_delimiter(text, minimum_chunk_size, chunk_delimiter))
    min_chunks = 1
    num_chunks = int(min_chunks + detail * (max_chunks - min_chunks))

    # adjusting the chunk_size based on interpolated number of chunks
    document_length = len(tokenize(text))
    chunk_size = max(minimum_chunk_size, document_length // num_chunks)
    text_chunks = chunk_on_delimiter(text, chunk_size, chunk_delimiter)
    if verbose:
        print(f"Splitting the text into {len(text_chunks)} chunks to be summarized.")
        print(f"Chunk lengths are {[len(tokenize(x)) for x in text_chunks]}")


    # setting system message
    system_message_content = "Rewrite this text in summarized form."
    if additional_instructions is not None:
        system_message_content += f"\n\n{additional_instructions}"

    accumulated_summaries = []
    for chunk in tqdm(text_chunks):
        if summarize_recursively and accumulated_summaries:
            # Creating a structured prompt for recursive summarization
            accumulated_summaries_string = '\n\n'.join(accumulated_summaries)
            user_message_content = f"Previous summaries:\n\n{accumulated_summaries_string}\n\nText to summarize next:\n\n{chunk}"
        else:
            # Directly passing the chunk for summarization without recursive context
            user_message_content = chunk

        # Constructing messages based on whether recursive summarization is applied
        messages = [
            {"role": "system", "content": system_message_content},
            {"role": "user", "content": user_message_content}
        ]

        # Assuming this function gets the completion and works as expected
        response = get_chat_completion(messages, model=model)
        accumulated_summaries.append(response)

    # Compiljng the final summary from partial summaries
    final_summary = '\n\n'.join(accumulated_summaries)

    return final_summary

## Level of Depthness with chunking [0, 0.25, 0.5, 1]

In [80]:
summary_with_detail_0 = summarize(text, detail=0, verbose=True)

Splitting the text into 1 chunks to be summarized.
Chunk lengths are [2380]


100%|██████████| 1/1 [00:10<00:00, 10.88s/it]


In [44]:
summary_with_detail_pt25 = summarize(text, detail=0.25, verbose=True)

Splitting the text into 2 chunks to be summarized.
Chunk lengths are [1190, 1190]


100%|██████████| 2/2 [00:16<00:00,  8.03s/it]


In [45]:
summary_with_detail_pt5 = summarize(text, detail=0.5, verbose=True)

Splitting the text into 4 chunks to be summarized.
Chunk lengths are [784, 779, 783, 34]


100%|██████████| 4/4 [00:30<00:00,  7.51s/it]


In [46]:
summary_with_detail_1 = summarize(text, detail=1, verbose=True)

Splitting the text into 5 chunks to be summarized.
Chunk lengths are [495, 480, 478, 495, 434]


100%|██████████| 5/5 [00:46<00:00,  9.37s/it]


## Length of summeries

In [47]:
[len(tokenize(x)) for x in
[summary_with_detail_0, summary_with_detail_pt25, summary_with_detail_pt5, summary_with_detail_1]]

[375, 439, 732, 1101]

## Summary based on detail

Details refer to specific, granular information or particular elements within the text
- higher detail results in longer and in-depth reviews
- lower detail results in shorter and quick overviews

## 0

In [48]:
print(summary_with_detail_0)

**Clients Module Summary**

**Date:** August 18th, 2023

**Overview:**
The Clients Module is designed to manage two types of clients: Individuals (including families and retail customers) and Groups (such as employers, companies, or associations). This module provides tools to manage client information, memberships, transactions, and more.

**Key Features:**

1. **Adding Clients:**
   - **Individual Clients:** Add through the Clients Module by filling out an online form with details like name, date of birth, and contact information.
   - **Group Clients:** Similar process to individual clients but includes group information and contact details.

2. **Managing Memberships:**
   - Approve or decline memberships.
   - Modify memberships including start and end dates.
   - Add or cancel memberships.

3. **Transactions and Billing:**
   - Add one-time transactions.
   - Update billing information and payment methods.
   - Send agreements for signatures.

4. **Client Status:**
   - Submitted

# 0.25

In [49]:
print(summary_with_detail_pt25 )

Clients Module Summary

August 18th, 2023

The Clients Module is designed to manage two types of clients: Individuals (including families and retail customers) and Groups (such as employers and companies). This system allows for efficient management of client information, including service details, memberships, transactions, and agreements.

Key Functions:
1. Adding Clients: Users can add individual clients or groups by filling out an online form and selecting the client type (Primary, Dependent, Group, Group Member).
2. Managing Memberships: Users can approve, decline, modify, or cancel memberships, and adjust membership dates.
3. Billing Updates: Billing information can be updated, and payment methods can be added or changed.
4. Client Documentation: Files can be uploaded and managed through the module, and client ID cards can be viewed if enabled.
5. Quick View: Provides a snapshot of client enrollments, transactions, and cases for easy reference.

Overall, the module streamlines th

# 0.5

In [50]:
print(summary_with_detail_pt5 )

**Clients Module Summary**

**Date:** August 18th, 2023

**Overview:**
The Clients module is designed to manage two types of clients: Individuals (including families and retail customers) and Groups (such as employers, companies, or associations).

**Functionalities:**
- **Manage Client Information:** Handle service/case details, membership approvals, transactions, agreements, and other management tasks.
- **Client Types:** Includes Primary, Dependent, Group, and Group Member.
- **Adding Clients:**
  - **Individual Clients:** Accessible via the Clients module, where users can add individuals by filling out an online form with personal and contact details, followed by billing and payment information.
  - **Dependents:** Added to a primary member by selecting the dependent option, filling out the required form, and completing billing details.
  - **Group Clients:** Similar process to adding individual clients but involves entering group information and contact details.
- **Client Status:

## 1

In [51]:
print(summary_with_detail_1)

Clients Module Summary

Date: August 18th, 2023

Overview

The Clients module is designed to manage two types of clients: Individuals and Groups. Individuals include single customers, families, or retail clients, while Groups consist of employers, companies, or associations.

Features

The module provides tools to manage client information, oversee service or case details, handle memberships (approvals and declines), manage transactions, send agreements, and perform other administrative tasks.

Instructions for Use

1. Adding Clients:
   - Individual Clients: Access the clients module, select the individual tab, and use the +Add button to fill out an online form with client details such as name, date of birth, and contact information. Choose service offerings and billing options.
   - Group Clients: Similar steps apply, with the addition of adding multiple group members.

2. Managing Memberships:
   - Edit client details, approve or decline memberships, modify membership dates, add or 

## Summary with additional instructions (prompts)

 Listing in points

## 0.1

In [52]:
summary_with_additional_instructions = summarize(text, detail=0.1,
                                                    additional_instructions="Write in point form and focus on numerical data.")
print(summary_with_additional_instructions)

100%|██████████| 1/1 [00:14<00:00, 14.17s/it]

- **Types of Clients**: Individuals (including families) and Groups (employers, companies, associations).
- **Client Management Tools**: Manage service/case information, memberships, transactions, agreements, and other tasks.
- **Adding Clients**:
  - Individual Client: Via Clients Module > Individual Tab > +Add button.
  - Dependent: Added to a primary member.
  - Group Client: Via Clients Module > Group Tab > +Add button.
  - Group Members: Added to existing groups.
- **Client Information**:
  - Four client types: Primary, Dependent, Group, Group Member.
  - Information required: Name, date of birth, email, phone, address.
  - Billing/payment setup included.
- **Client Status**:
  - Default: "Submitted" (charged upon "Approved").
  - Other statuses: Active, Inactive, Canceled, Declined.
- **Client Functions**:
  - Edit details, approve/decline/modify membership, add/change membership dates, add transactions, send agreements.
- **Membership Management**:
  - Approve/decline membership

## 0.6

In [53]:
summary_with_additional_instructions = summarize(text, detail=0.6,
                                                    additional_instructions="Write in point form and focus on numerical data.")
print(summary_with_additional_instructions)

100%|██████████| 4/4 [00:30<00:00,  7.72s/it]

- **Types of Clients**: Individuals (including families, retail customers) and Groups (employers, companies, associations).
- **Client Management Tools**: Manage service/case information, memberships, transactions, agreements, and other tasks.
- **Client Addition Process**:
  - **Individual Client**: Add via Clients Module > Individual Tab > +Add button > Fill online form (Primary, Dependent) > Save > Select offering and start date > Billing/payment options.
  - **Group Client**: Add via Clients Module > Group Tab > +Add button > Fill online form (Group, Group Member) > Save > Select membership offering and start date > Billing/payment options.
- **Client Modifications**:
  - Edit details, approve/decline/modify membership, change membership dates, add/cancel membership.
  - Add one-time transactions, send agreements, delete members.
- **Client Status**: Default is “Submitted”; charged once “Approved”.
- **File Management**: Upload, download, rename, and view files using the +Upload fi

## 1

In [54]:
summary_with_additional_instructions = summarize(text, detail=1,
                                                    additional_instructions="Write in point form and focus on numerical data.")
print(summary_with_additional_instructions)

100%|██████████| 5/5 [00:57<00:00, 11.47s/it]

- Types of clients: Individuals (individual customers, families, retail customers) and Groups (employers, companies, associations).
- Management tools: Manage client information, service/case details, memberships, transactions, agreements.
- Client operations:
  - Add Individual Client
  - Add dependents to a primary member
  - Add a Group Client
  - Add members to a group
  - Quick view of group members/dependents, billings, cases
  - Update Billing
- Client ellipsis functions:
  - Edit client details
  - Approve/Decline/Modify membership
  - Change membership start/end date
  - Add/Cancel additional membership
  - Add a one-time transaction
  - Send an agreement
  - Delete member
- Client status: "Submitted" by default, charged once "Approved"
- Adding an Individual Client:
  - Access through clients module > individual tab
  - Use +Add button, fill online form (Primary, Dependent, Group, Group Member)
  - Enter personal details (Name, DOB, email, phone, address)
  - Select service o

Detail summary with prompts

In [55]:
summary_with_additional_instructions = summarize(text, detail=0.1,
                                                    additional_instructions="Summarize the key details of this admin and client portal of Vitafy Healthcare system.")
print(summary_with_additional_instructions)

100%|██████████| 1/1 [00:09<00:00,  9.34s/it]

The Vitafy Healthcare system's admin and client portal allows for efficient management of individual and group clients. Clients are categorized into Individuals (including families or retail customers) and Groups (such as employers or companies). The system provides tools to manage client information, service details, memberships, transactions, and agreements.

Key functionalities include:
- Adding individual clients and dependents.
- Setting up groups or companies and adding members to these groups.
- Viewing quick details of group members, billings, and cases.
- Updating billing information and managing various payment methods.
- Utilizing the client ellipsis function to edit client details, approve or decline memberships, modify memberships, and handle other membership-related tasks such as changing start and end dates, adding or canceling memberships.
- Adding one-time transactions and sending agreements.
- Managing client statuses which include Submitted, Approved, Canceled, Decli

In [56]:
summary_with_additional_instructions = summarize(text, detail=0.9,
                                                    additional_instructions="Summarize the key details of this admin and client portal of Vitafy Healthcare system.")
print(summary_with_additional_instructions)

100%|██████████| 5/5 [00:38<00:00,  7.75s/it]

The Clients Module of Vitafy Healthcare system, updated on August 18th, 2023, allows for efficient management of two types of clients: Individuals (including families or retail customers) and Groups (such as employers, companies, or associations). The module provides tools to manage client information, service/case details, memberships, transactions, and agreements.

Key functionalities include:
- Adding individual clients and dependents.
- Adding group clients and their members.
- Viewing group members, billings, and cases.
- Updating billing information.
- Editing client details, membership approvals, modifications, and cancellations.
- Adding one-time transactions and sending agreements.
- Deleting members and managing client statuses.

To add an individual client, navigate to the clients module, select the individual tab, and fill out the online form with details like name, date of birth, and contact information. Payment details are added in the billing section, and files can be up

## Recursive Summarization

In [57]:
recursive_summary = summarize(text, detail=0.1, summarize_recursively=True)
print(recursive_summary)

100%|██████████| 1/1 [00:13<00:00, 13.41s/it]

**Clients Module Summary**

**Date:** August 18th, 2023

**Overview:**
The Clients Module is designed to manage two types of clients: Individuals (including families and retail customers) and Groups (such as employers and companies). This system facilitates various administrative tasks including managing client information, memberships, transactions, and agreements.

**Key Features:**

1. **Adding Clients:**
   - **Individual Clients:** Can be added through the module by filling out an online form specifying details like name, contact, and billing information.
   - **Group Clients:** Similar process to individuals but involves entering group-specific information.

2. **Managing Memberships:**
   - Memberships can be approved or declined, with the ability to modify, add, or cancel memberships. Changes to the start and end dates of memberships can also be managed.

3. **Transactions and Agreements:**
   - One-time transactions can be added manually.
   - Agreements can be sent to clients

In [58]:
recursive_summary = summarize(text, detail=0.9, summarize_recursively=True)
print(recursive_summary)

100%|██████████| 5/5 [01:00<00:00, 12.16s/it]

**Clients Module Summary**

**Date:** August 18th, 2023

**Overview:**
The Clients Module is designed to manage two types of clients: Individuals (including families and retail customers) and Groups (such as employers, companies, or associations). This module provides tools to manage client information, transactions, memberships, and billing efficiently.

**Key Features:**

1. **Managing Client Information:**
   - Oversee service/case details, memberships, and transactions.
   - Send agreements and perform other management tasks.

2. **Adding Clients:**
   - **Individual Clients:** Accessible via the clients module, where details like name, date of birth, and contact information are entered. Options to add dependents are available.
   - **Group Clients:** Similar access point, with functionality to add multiple group members.

3. **Membership Management:**
   - Approve or decline memberships.
   - Modify membership details including start and end dates.
   - Add or cancel memberships.


## Recursive summarization with prompts

In [59]:
recursive_summary = summarize(text, detail=0.9, additional_instructions="Summarize the key details of the Admin and Client portal of Vitafy Healthcare system.", summarize_recursively=True)
print(recursive_summary)

100%|██████████| 5/5 [00:41<00:00,  8.21s/it]

The Vitafy Healthcare system's Admin and Client portal allows for efficient management of both individual and group clients. Individual clients include single customers, families, or retail customers, while group clients consist of employers, companies, or associations. The system provides tools to manage client information, service details, memberships, and transactions.

Key functionalities include:
- Adding individual and group clients.
- Managing dependents for primary members.
- Viewing group members, billings, and cases.
- Updating billing information.
- Editing client details, approving or declining memberships, modifying membership terms, and handling other membership-related tasks.
- Adding one-time transactions and sending agreements.
- Managing client statuses, with new clients being "Submitted" by default and only charged upon approval.

The process to add an individual client involves selecting the client type, filling out personal and contact details, choosing service off

In [60]:
recursive_summary = summarize(text, detail=0.1, additional_instructions="Summarize the key details of the Admin and Client portal of Vitafy Healthcare system.", summarize_recursively=True)
print(recursive_summary)

100%|██████████| 1/1 [00:08<00:00,  8.11s/it]

The Vitafy Healthcare system provides a comprehensive Clients Module designed to manage both individual and group clients efficiently. The module allows for the addition and management of individual clients, including families and retail customers, as well as groups such as employers and companies.

Key functionalities include:
- Adding and managing individual and group clients.
- Overseeing client service and case information.
- Approving or declining memberships and transactions.
- Sending agreements and managing billing details.
- Editing client details, modifying memberships, and updating billing information.
- Adding dependents to primary members and members to groups.
- Quick viewing of group members, dependents, billings, and cases.
- Uploading and managing files related to clients.
- Viewing client ID cards and managing client statuses, which include "Submitted," "Approved," "Active," "Inactive," "Canceled," and "Declined."

The system also allows for the activation of client a

In [61]:
recursive_summary = summarize(text, detail=0.5, additional_instructions="Summarize the key details of the Admin and Client portal of Vitafy Healthcare system in paragraphs not in lists", summarize_recursively=True)
print(recursive_summary)

100%|██████████| 4/4 [00:45<00:00, 11.44s/it]

The Vitafy Healthcare system's Clients Module is designed to manage two main types of clients: Individuals and Groups. Individuals include single customers, families, or retail clients, while Groups consist of employers, companies, or associations. The system provides tools to manage client information, oversee service or case details, handle memberships, and conduct transactions. Users can add both individual clients and groups, along with dependents or group members, and manage various aspects such as billing, membership approvals, and document uploads.

For individual clients, the process involves selecting the client type (Primary or Dependent), filling out necessary personal and contact details, and choosing service offerings and billing options. Similarly, adding a group involves inputting group details, selecting membership types, and setting up billing information. Both processes allow for the uploading and management of files and viewing of client ID cards if enabled. The syst

In [63]:
recursive_summary = summarize(text, detail=0.3, additional_instructions="Summarize the key details of the Admin and Client portal of Vitafy Healthcare system in a paragraph", summarize_recursively=True)
print(recursive_summary)

100%|██████████| 2/2 [00:11<00:00,  5.94s/it]

The Vitafy Healthcare system's Admin and Client portal allows efficient management of individual and group clients, including families, employers, and companies. Users can add and manage client information, such as service details, memberships, and transactions. The system supports adding individual clients and dependents, as well as groups and their members, with functionalities to approve or decline memberships, update billing, and send agreements. Each client type—primary, dependent, group, and group member—can be added through a form that captures essential details and payment information. Files can be uploaded and managed, and client status is tracked, with approvals required for activating accounts. The portal also offers a quick view for an overview of enrollments, services, and billing, enhancing the management of client relationships and administrative tasks.

The Vitafy Healthcare system's Admin and Client portal offers a comprehensive suite of functionalities to manage clien

In [64]:
recursive_summary = summarize(text, detail=0.1, additional_instructions="Summarize the key details of the Admin and Client portal of Vitafy Healthcare system in a paragraph", summarize_recursively=True)
print(recursive_summary)

100%|██████████| 1/1 [00:07<00:00,  7.81s/it]

The Vitafy Healthcare system's Admin and Client portal allows efficient management of individual and group clients, including families, employers, and companies. The system facilitates the addition of individual clients and groups, managing dependents, and overseeing memberships and transactions. Users can update client information, approve or decline memberships, modify membership details, and handle billing and payments. The portal also supports sending agreements, adding one-time transactions, and uploading necessary files. Client statuses such as "Submitted," "Approved," "Active," and "Inactive" help track the progress and current state of each client. Additional functionalities include viewing client ID cards, quick access to client details, and the ability to send invites for account activation. The system is designed to streamline the administrative tasks associated with managing healthcare services and memberships, ensuring a smooth operation and client management process.


## Combining Pinecone & OpenAI model

In [100]:
summary = chain.run(input_documents=search, detail=0.1, question="Summarize the key details of the Admin and Client portal of Vitafy Healthcare system in a paragraph.", summarize_recursively = True)
print(summary)

/Users/user/.pyenv/versions/3.12.3/lib/python3.12/site-packages/langchain_openai/chat_models/base.py:357: UserWarning: Unexpected type for token usage: <class 'NoneType'>
  warnings.warn(f"Unexpected type for token usage: {type(new_usage)}")


The system allows for the uploading, downloading, renaming, and viewing of files. Client ID cards can be viewed in the ID tab if enabled. Clients are initially marked as "Submitted" and will only be charged once approved. Dependents can be added to primary clients, who are individuals connected through family ties or association. The client details can be viewed by clicking on the + sign on the left-hand side, showing membership, billing, and cases information. The system also allows for setting up groups or companies, editing client profiles, updating billing details, and uploading files.


## Tracking token & cost usage 


In [108]:
from langchain_community.callbacks import get_openai_callback

with get_openai_callback() as cb:
    summary = chain.run(input_documents=search, detail=1, question="Summarize the key details of the Admin and Client portal of Vitafy Healthcare system in a paragraph.", summarize_recursively = True)
print(summary, cb)


/Users/user/.pyenv/versions/3.12.3/lib/python3.12/site-packages/langchain_openai/chat_models/base.py:357: UserWarning: Unexpected type for token usage: <class 'NoneType'>
  warnings.warn(f"Unexpected type for token usage: {type(new_usage)}")


The system allows users to upload, download, rename, and view files. Client ID cards can be viewed in the ID tab if enabled. Clients are initially marked as "Submitted" and will only be charged once approved. Dependents can be added to primary clients. Client details can be viewed by clicking on the + sign. The system also allows for setting up groups or companies by filling out an online form. Client Ellipsis functions provide additional functionalities for editing profile information, viewing membership status, updating billing details, and uploading files. Tokens Used: 1012
	Prompt Tokens: 690
	Completion Tokens: 322
Successful Requests: 5
Total Cost (USD): $0.000828


In [ ]:
🦜️🏓 LangServe